<a href="https://colab.research.google.com/github/belellei/data_enginner_study/blob/main/base_inicial_ifdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instituições


In [ ]:
# Instala a biblioteca do BigQuery
!pip install --quiet google-cloud-bigquery

# Importa as bibliotecas necessárias
from google.colab import files
from google.cloud import bigquery
from google.colab import files
import requests
import sys
import os
import requests
from datetime import datetime

# Definir o caminho fixo para o arquivo de credenciais no Colab
credential_path_colab = "/content/credentials.json"

# Nova seção

In [ ]:


# Fazer upload do arquivo de credenciais JSON do Google Cloud
print("Por favor, faça o upload do arquivo de credenciais JSON do Google Cloud.")
uploaded = files.upload()

# Extrair o nome do arquivo de credenciais
credential_filename = list(uploaded.keys())[0]

# Mover o arquivo de credenciais para um local fixo no Colab
credential_path = "/content/credentials.json"
os.rename(credential_filename, credential_path)

# Definir a variável de ambiente com o caminho para o arquivo JSON das credenciais
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path

print(f"Credenciais armazenadas em: {credential_path}")


Por favor, faça o upload do arquivo de credenciais JSON do Google Cloud.


Saving genial-insight-433719-c4-bd82edde8bcb.json to genial-insight-433719-c4-bd82edde8bcb.json
Credenciais armazenadas em: /content/credentials.json


In [ ]:
import os
import sys
import requests
from google.cloud import bigquery
from datetime import datetime

# Definir o caminho fixo para o arquivo de credenciais no Colab
credential_path_colab = "/content/credentials.json"

# Verifica se o arquivo de credenciais existe no local especificado
if not os.path.exists(credential_path_colab):
    print(f"Arquivo de credenciais não encontrado em {credential_path_colab}. Verifique se ele foi salvo corretamente.")
    sys.exit(1)

# Define a variável de ambiente com o caminho para o arquivo JSON das credenciais
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path_colab

# Função para conectar ao BigQuery
def conectar_ao_bigquery():
    try:
        client = bigquery.Client()
        return client
    except Exception as e:
        print(f"Erro ao conectar ao BigQuery: {e}")
        sys.exit(1)

# Conexão com o BigQuery
client = conectar_ao_bigquery()

# Função para criar dataset e tabela se não existirem
def criar_dataset_e_tabela(dataset_id, table_id):
    dataset_ref = client.dataset(dataset_id)

    # Cria o dataset se ele não existir
    try:
        client.get_dataset(dataset_ref)
        print(f"Dataset {dataset_id} já existe.")
    except:
        dataset = bigquery.Dataset(dataset_ref)
        dataset = client.create_dataset(dataset)
        print(f"Dataset {dataset_id} criado.")

    # Define o esquema da tabela
    schema = [
        bigquery.SchemaField("cod_instituicao", "STRING"),
        bigquery.SchemaField("data", "DATE"),
        bigquery.SchemaField("nome_instituicao", "STRING"),
        bigquery.SchemaField("data_inicio_atividade", "DATE"),
        bigquery.SchemaField("tipo_controle_bancario", "STRING"),
        bigquery.SchemaField("tipo_documento", "STRING"),
        bigquery.SchemaField("codigo_controle", "STRING"),
        bigquery.SchemaField("segmento", "STRING"),
        bigquery.SchemaField("atividade", "STRING"),
        bigquery.SchemaField("uf", "STRING"),
        bigquery.SchemaField("municipio", "STRING"),
        bigquery.SchemaField("superintendencia_regional", "STRING"),
        bigquery.SchemaField("cod_conglomerado_financeiro", "STRING"),
        bigquery.SchemaField("cod_conglomerado_prudencial", "STRING"),
        bigquery.SchemaField("cnpj_instituicao_lider", "STRING"),
        bigquery.SchemaField("situacao", "STRING"),
    ]

    table_ref = dataset_ref.table(table_id)

    # Cria a tabela se ela não existir
    try:
        client.get_table(table_ref)
        print(f"Tabela {table_id} já existe.")
    except:
        table = bigquery.Table(table_ref, schema=schema)
        table = client.create_table(table)
        print(f"Tabela {table_id} criada.")

# Chamar a função para criar o dataset e a tabela
dataset_id = "br_bcb_ifdata"
table_id = "instituicoes_bancarias_brasileiras"
criar_dataset_e_tabela(dataset_id, table_id)

# Função para formatar datas
def formatar_data(data_str):
    try:
        if data_str is None:
            return None
        if isinstance(data_str, int):
            data_str = str(data_str)
        if len(data_str) == 6:  # Formato 'YYYYMM'
            data_str = f"{data_str[:4]}-{data_str[4:]}-01"
        if "T" in data_str:
            data_str = data_str.split("T")[0]
        formatted_date = datetime.strptime(data_str, "%Y-%m-%d").date()
        return formatted_date.isoformat()  # Converte para string no formato ISO
    except (ValueError, TypeError) as e:
        print(f"Erro ao formatar data: {data_str} - Erro: {e}")
        return None

# Função para inserir dados no BigQuery
def inserir_dados_no_bigquery(dados):
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)

    rows_to_insert = [
        {
            "cod_instituicao": item.get('CodInst'),
            "data": formatar_data(item.get('Data')),
            "nome_instituicao": item.get('NomeInstituicao'),
            "data_inicio_atividade": formatar_data(item.get('DataInicioAtividade')),
            "tipo_controle_bancario": item.get('Tcb'),
            "tipo_documento": item.get('Td'),
            "codigo_controle": item.get('Tc'),
            "segmento": item.get('SegmentoTb'),
            "atividade": item.get('Atividade'),
            "uf": item.get('Uf'),
            "municipio": item.get('Municipio'),
            "superintendencia_regional": item.get('Sr'),
            "cod_conglomerado_financeiro": item.get('CodConglomeradoFinanceiro'),
            "cod_conglomerado_prudencial": item.get('CodConglomeradoPrudencial'),
            "cnpj_instituicao_lider": item.get('CnpjInstituicaoLider'),
            "situacao": item.get('Situacao'),
        }
        for item in dados
    ]

    errors = client.insert_rows_json(table, rows_to_insert)
    if errors == []:
        print("Dados inseridos com sucesso no BigQuery")
    else:
        for error in errors:
            print(f"Erro ao inserir os dados (index {error['index']}): {error['errors']}")

# Função para buscar dados da API e inserir no BigQuery
def buscar_e_inserir_dados():
    url = "https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/IfDataCadastro(AnoMes=@AnoMes)"
    meses_disponiveis = [1, 3, 6, 9, 12]

    for ano in range(2002, 2025):
        for mes in meses_disponiveis:
            ano_mes = f"{ano}{mes:02d}"
            print(f"Buscando dados para {ano_mes}...")

            params = {
                '@AnoMes': ano_mes,
                '$top': 10000,
                '$format': 'json',
                '$select': 'CodInst,Data,NomeInstituicao,DataInicioAtividade,Tcb,Td,Tc,SegmentoTb,Atividade,Uf,Municipio,Sr,CodConglomeradoFinanceiro,CodConglomeradoPrudencial,CnpjInstituicaoLider,Situacao'
            }

            try:
                response = requests.get(url, params=params)
                response.raise_for_status()
                dados = response.json().get('value', [])

                if dados:
                    inserir_dados_no_bigquery(dados)
                    print(f"{len(dados)} registros inseridos para {ano_mes}")
                else:
                    print(f"Nenhum dado encontrado para {ano_mes}")

            except requests.exceptions.HTTPError as http_err:
                print(f"Erro HTTP: {http_err}")
            except Exception as err:
                print(f"Outro erro: {err}")

# Executa o processo de busca e inserção
buscar_e_inserir_dados()

Dataset br_bcb_ifdata já existe.
Tabela instituicoes_bancarias_brasileiras já existe.
Buscando dados para 200201...


KeyboardInterrupt: 

# Lista de Relatórios

In [ ]:

# Função para criar dataset e tabela se não existirem
def criar_dataset_e_tabela(dataset_id, table_id):
    dataset_ref = client.dataset(dataset_id)

    # Cria o dataset se não existir
    try:
        client.get_dataset(dataset_ref)
        print(f"Dataset {dataset_id} já existe.")
    except:
        dataset = bigquery.Dataset(dataset_ref)
        dataset = client.create_dataset(dataset)
        print(f"Dataset {dataset_id} criado.")

    # Define o esquema da tabela com as colunas 'nome_relatorio' e 'numero_relatorio'
    schema = [
        bigquery.SchemaField("nome_relatorio", "STRING"),
        bigquery.SchemaField("numero_relatorio", "STRING"),
    ]

    table_ref = dataset_ref.table(table_id)

    # Cria a tabela se não existir
    try:
        client.get_table(table_ref)
        print(f"Tabela {table_id} já existe.")
    except:
        table = bigquery.Table(table_ref, schema=schema)
        table = client.create_table(table)
        print(f"Tabela {table_id} criada.")

# Chamar a função para criar o dataset e a tabela
dataset_id = "br_bcb_ifdata"
table_id = "lista_relatorios"
criar_dataset_e_tabela(dataset_id, table_id)

# Função para inserir dados no BigQuery
def inserir_dados_no_bigquery(dados):
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)

    # Inserir somente as colunas 'nome_relatorio' e 'numero_relatorio'
    rows_to_insert = [
        {
            "nome_relatorio": item.get('NomeRelatorio'),
            "numero_relatorio": item.get('NumeroRelatorio'),
        }
        for item in dados
    ]

    errors = client.insert_rows_json(table, rows_to_insert)
    if errors == []:
        print("Dados inseridos com sucesso no BigQuery")
    else:
        print(f"Erros ao inserir os dados: {errors}")

# Função para buscar dados da API e inserir no BigQuery
def buscar_e_inserir_dados():
    url = "https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/ListaDeRelatorio()"

    # Adicionando os parâmetros corretos para a requisição
    params = {
        "$top": 10000,
        "$skip": 0,
        "$format": "json",
        "$select": "NomeRelatorio,NumeroRelatorio"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        dados = response.json().get('value', [])

        if dados:
            inserir_dados_no_bigquery(dados)
            print(f"{len(dados)} registros inseridos.")
        else:
            print(f"Nenhum dado encontrado.")

    except requests.exceptions.HTTPError as http_err:
        print(f"Erro HTTP: {http_err}")
    except Exception as err:
        print(f"Outro erro: {err}")

# Executar a função para buscar e inserir os dados
buscar_e_inserir_dados()


Dataset br_bcb_ifdata já existe.
Tabela lista_relatorios criada.
Dados inseridos com sucesso no BigQuery
15 registros inseridos.


# Base dos dados

In [ ]:

from google.cloud import bigquery
import os
import sys

# Definir o caminho fixo para o arquivo de credenciais no Colab
credential_path_colab = "/content/credentials.json"

# Verifica se o arquivo de credenciais existe no local especificado
if not os.path.exists(credential_path_colab):
    print(f"Arquivo de credenciais não encontrado em {credential_path_colab}. Verifique se ele foi salvo corretamente.")
    sys.exit(1)

# Define a variável de ambiente com o caminho para o arquivo JSON das credenciais
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path_colab

# Função para conectar ao BigQuery
def conectar_ao_bigquery():
    try:
        client = bigquery.Client()
        return client
    except Exception as e:
        print(f"Erro ao conectar ao BigQuery: {e}")
        sys.exit(1)

# Conexão com o BigQuery
client = conectar_ao_bigquery()


# Função para criar dataset e tabela se não existirem
def criar_dataset_e_tabela(dataset_id, table_id):
    dataset_ref = client.dataset(dataset_id)

    # Cria o dataset se não existir
    try:
        client.get_dataset(dataset_ref)
        print(f"Dataset {dataset_id} já existe.")
    except:
        dataset = bigquery.Dataset(dataset_ref)
        dataset = client.create_dataset(dataset)
        print(f"Dataset {dataset_id} criado.")

    # Define o esquema da tabela com todos os campos
    schema = [
        bigquery.SchemaField("tipo_instituicao", "STRING"),
        bigquery.SchemaField("cod_instituicao", "STRING"),
        bigquery.SchemaField("ano_mes", "STRING"),
        bigquery.SchemaField("nome_relatorio", "STRING"),
        bigquery.SchemaField("numero_relatorio", "STRING"),
        bigquery.SchemaField("grupo", "STRING"),
        bigquery.SchemaField("conta", "STRING"),
        bigquery.SchemaField("nome_coluna", "STRING"),
        bigquery.SchemaField("descricao_coluna", "STRING"),
        bigquery.SchemaField("saldo", "FLOAT")
    ]

    table_ref = dataset_ref.table(table_id)

    # Cria a tabela se não existir
    try:
        client.get_table(table_ref)
        print(f"Tabela {table_id} já existe.")
    except:
        table = bigquery.Table(table_ref, schema=schema)
        table = client.create_table(table)
        print(f"Tabela {table_id} criada.")

# Chamar a função para criar o dataset e a tabela
dataset_id = "br_bcb_ifdata"
table_id = "ifdata_valores_completos"
criar_dataset_e_tabela(dataset_id, table_id)


# Função para inserir dados no BigQuery (manter saldo como FLOAT)
def inserir_dados_no_bigquery(dados, relatorio, tipo_instituicao):
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)

    rows_to_insert = [
        {
            "tipo_instituicao": item.get('TipoInstituicao'),
            "cod_instituicao": item.get('CodInst'),
            "ano_mes": item.get('AnoMes'),
            "nome_relatorio": item.get('NomeRelatorio'),
            "numero_relatorio": item.get('NumeroRelatorio'),
            "grupo": item.get('Grupo'),
            "conta": item.get('Conta'),
            "nome_coluna": item.get('NomeColuna'),
            "descricao_coluna": item.get('DescricaoColuna'),
            # Manter o saldo como FLOAT sem formatação
            "saldo": item.get('Saldo')
        }
        for item in dados
    ]

    errors = client.insert_rows_json(table, rows_to_insert)
    if errors == []:
        print("Dados inseridos com sucesso no BigQuery")
    else:
        print(f"Erros ao inserir os dados: {errors}")




# Função para buscar dados da API e inserir no BigQuery
def buscar_e_inserir_dados():
    url_base = "https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/IfDataValores(AnoMes=@AnoMes,TipoInstituicao=@TipoInstituicao,Relatorio=@Relatorio)"
    tipos_instituicao = [1, 2, 3, 4]
    meses_disponiveis = [1, 3, 6, 9, 12]

    for ano in range(2002, 2025):
        for mes in meses_disponiveis:
            ano_mes = f"{ano}{mes:02d}"
            for tipo_instituicao in tipos_instituicao:
                print(f"Buscando dados para {ano_mes}, tipo de instituição {tipo_instituicao}...")

                params = {
                    "@AnoMes": ano_mes,  # AnoMes formatado como string (YYYYMM)
                    "@TipoInstituicao": tipo_instituicao,  # TipoInstituicao como número
                    "@Relatorio": "'T'",  # Relatorio é uma string delimitada por aspas simples
                    "$top": 10000,  # Limitar a 100 registros
                    "$skip": 0,  # Ignorar 0 registros
                    "$format": "json",  # Resposta em formato JSON
                    "$select": "TipoInstituicao,CodInst,AnoMes,NomeRelatorio,NumeroRelatorio,Grupo,Conta,NomeColuna,DescricaoColuna,Saldo"
                }

                try:
                    response = requests.get(url_base, params=params)
                    response.raise_for_status()

                    # Exibir a URL gerada para depuração
                    print(f"URL gerada: {response.url}")

                    dados = response.json().get('value', [])

                    if dados:
                        inserir_dados_no_bigquery(dados, "Relatorio", tipo_instituicao)
                        print(f"{len(dados)} registros inseridos para {ano_mes}, tipo {tipo_instituicao}.")
                    else:
                        print(f"Nenhum dado encontrado para {ano_mes}, tipo {tipo_instituicao}.")

                except requests.exceptions.HTTPError as http_err:
                    print(f"Erro HTTP: {http_err}")
                except Exception as err:
                    print(f"Outro erro: {err}")

# Executar a função para buscar e inserir os dados
buscar_e_inserir_dados()

Dataset br_bcb_ifdata já existe.
Tabela ifdata_valores_completos já existe.
Buscando dados para 200201, tipo de instituição 1...
URL gerada: https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/IfDataValores(AnoMes=@AnoMes,TipoInstituicao=@TipoInstituicao,Relatorio=@Relatorio)?%40AnoMes=200201&%40TipoInstituicao=1&%40Relatorio=%27T%27&%24top=10000&%24skip=0&%24format=json&%24select=TipoInstituicao%2CCodInst%2CAnoMes%2CNomeRelatorio%2CNumeroRelatorio%2CGrupo%2CConta%2CNomeColuna%2CDescricaoColuna%2CSaldo
Nenhum dado encontrado para 200201, tipo 1.
Buscando dados para 200201, tipo de instituição 2...
URL gerada: https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/IfDataValores(AnoMes=@AnoMes,TipoInstituicao=@TipoInstituicao,Relatorio=@Relatorio)?%40AnoMes=200201&%40TipoInstituicao=2&%40Relatorio=%27T%27&%24top=10000&%24skip=0&%24format=json&%24select=TipoInstituicao%2CCodInst%2CAnoMes%2CNomeRelatorio%2CNumeroRelatorio%2CGrupo%2CConta%2CNomeColuna%2CDescricaoColuna%

# Tabela Dicionário

In [ ]:
table_id = "informacoes_bancarias"
project_id = "genial-insight-433719-c4"
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Cria a tabela no BigQuery
schema = [
    bigquery.SchemaField("tipo", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("codigo", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("descricao", "STRING", mode="REQUIRED")
]

table = bigquery.Table(table_ref, schema=schema)
client.create_table(table, exists_ok=True)

# Insere os dados na tabela
rows_to_insert = [
    {"tipo": "Controle Bancário", "codigo": "B1", "descricao": "Instituição individual do tipo Banco Comercial, Banco Múltiplo com Carteira Comercial ou caixas econômicas e Conglomerado composto de pelo menos uma instituição do tipo Banco Comercial, Banco Múltiplo com Carteira Comercial ou caixas econômicas."},
    {"tipo": "Controle Bancário", "codigo": "B2", "descricao": "Instituição individual do tipo Banco Múltiplo sem Carteira Comercial ou Banco de Câmbio ou Banco de Investimento e Conglomerado composto de pelo menos uma instituição do tipo Banco Múltiplo sem Carteira Comercial ou Banco de Investimento, mas sem conter instituições do tipo Banco Comercial e Banco Múltiplo com Carteira Comercial."},
    {"tipo": "Controle Bancário", "codigo": "B3S", "descricao": "Cooperativa de Crédito Singular."},
    {"tipo": "Controle Bancário", "codigo": "B3C", "descricao": "Central e Confederação de Cooperativas de Crédito."},
    {"tipo": "Controle Bancário", "codigo": "B4", "descricao": "Banco de Desenvolvimento."},
    {"tipo": "Controle Bancário", "codigo": "N1", "descricao": "Instituição não bancária atuante no mercado de crédito."},
    {"tipo": "Controle Bancário", "codigo": "N2", "descricao": "Instituição não bancária atuante no mercado de capitais."},
    {"tipo": "Controle Bancário", "codigo": "N4", "descricao": "Instituições de pagamento."},
    {"tipo": "Segmento Resolução 4.553/2017", "codigo": "S1", "descricao": "Bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas que (i) tenham porte (Exposição/Produto Interno Bruto) superior a 10%; ou (ii) exerçam atividade internacional relevante (ativos no exterior superiores a US$ 10 bilhões)."},
    {"tipo": "Segmento Resolução 4.553/2017", "codigo": "S2", "descricao": "Composto por: (i) bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas de porte inferior a 10% e igual ou superior a 1%; e (ii) demais instituições autorizadas a funcionar pelo Banco Central do Brasil de porte igual ou superior a 1% do PIB."},
    {"tipo": "Segmento Resolução 4.553/2017", "codigo": "S3", "descricao": "Instituições de porte inferior a 1% e igual ou superior a 0,1%."},
    {"tipo": "Segmento Resolução 4.553/2017", "codigo": "S4", "descricao": "Instituições de porte inferior a 0,1%."},
    {"tipo": "Segmento Resolução 4.553/2017", "codigo": "S5", "descricao": "Composto por: (i) instituições de porte inferior a 0,1% que utilizem metodologia facultativa simplificada para apuração dos requerimentos mínimos de Patrimônio de Referência (PR), de Nível I e de Capital Principal, exceto bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas; e (ii) não sujeitas a apuração de PR."},
    {"tipo": "Tipo de Documento", "codigo": "I", "descricao": "Instituição Independente."},
    {"tipo": "Tipo de Documento", "codigo": "C", "descricao": "Conglomerado."},
    {"tipo": "Controle", "codigo": "1", "descricao": "Público."},
    {"tipo": "Controle", "codigo": "2", "descricao": "Privado Nacional."},
    {"tipo": "Controle", "codigo": "3", "descricao": "Privado com Controle Estrangeiro."},
    {"tipo": "Situação", "codigo": "A", "descricao": "Em atividade."},
    {"tipo": "Situação", "codigo": "I", "descricao": "Inativa."}
]

errors = client.insert_rows_json(table, rows_to_insert)

if errors:
    print(f"Erros ao inserir linhas: {errors}")
else:
    print("Dados inseridos com sucesso na tabela informacoes_bancarias.")

Dados inseridos com sucesso na tabela informacoes_bancarias.
